## Part 1: Preprocessing

In [141]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [142]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [143]:
#Create 'y_df' with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [144]:
# Create a list of at least 10 column names to use as X data
columns = ['Age', 'StockOptionLevel', 'DistanceFromHome', 'Education', 'YearsAtCompany', 'HourlyRate', 'YearsSinceLastPromotion', 'NumCompaniesWorked', 'JobSatisfaction', 'OverTime']


# Create X_df using your selected columns
X_df = attrition_df[columns]


# Show the data types for X_df

X_df.dtypes

,0
Age,int64
StockOptionLevel,int64
DistanceFromHome,int64
Education,int64
YearsAtCompany,int64
HourlyRate,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64
JobSatisfaction,int64
OverTime,object


In [145]:
# Split the data into training and testing sets
#from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)

In [146]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

#import LabelEncoder from sklearn
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()

#Fit_Transform 'OverTime' column to change to numeric values

X_train['OverTime'] = le.fit_transform(X_train['OverTime'])
X_test['OverTime'] = le.fit_transform(X_test['OverTime'])


#Check to see if data types are all numeric
X_train.dtypes
X_test.dtypes

,0
Age,int64
StockOptionLevel,int64
DistanceFromHome,int64
Education,int64
YearsAtCompany,int64
HourlyRate,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64
JobSatisfaction,int64
OverTime,int64


In [147]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [148]:
#ONE HOT ENCODING TEST FOR DEPARTMENT
from sklearn.preprocessing import OneHotEncoder

department_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Fit the encoder to the training data
department_ohe.fit(y_train['Department'].values.reshape(-1, 1))

y_department_train_encoded = department_ohe.transform(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data

y_department_test_encoded = department_ohe.transform(y_test['Department'].values.reshape(-1, 1))


y_department_train_encoded[:5]

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [149]:
# Create a OneHotEncoder for the Attrition column

#Created Label Encoder instead for this data

from sklearn.preprocessing import LabelEncoder

attrition_le = LabelEncoder()

# Create two new variables by applying the encoder
# to the training and testing data

y_attrition_train_encoded = attrition_le.fit_transform(y_train['Attrition']).reshape(-1, 1)

y_attrition_test_encoded = attrition_le.transform(y_test['Attrition']).reshape(-1, 1)


## Create, Compile, and Train the Model

In [150]:
# Find the number of columns in the X training data
X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(X_df.shape[1],),name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

In [151]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer

department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [152]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer

attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [153]:
# Create the model
model = Model(inputs = input_layer, outputs = [attrition_output, department_output])

# Compile the model

model.compile(optimizer='adam', loss= {'attrition_output': 'binary_crossentropy', 'department_output': 'categorical_crossentropy'}, metrics = {'attrition_output': 'accuracy', 'department_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_24 (Dense)          │ (None, 64)             │            704 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_25 (Dense)          │ (None, 128)            │          8,320 │ dense_24[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, 32)             │          4,128 │ dense_25[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_26 (Dense)          │ (None, 32)             │          4,128 │ dense_25[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ dense_27[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_26[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,412 (68.02 KB)

 Trainable params: 17,412 (68.02 KB)

 Non-trainable params: 0 (0.00 B)

In [154]:
# Train the model

history = model.fit(X_train_scaled,{'attrition_output': y_attrition_train_encoded, 'department_output': y_department_train_encoded}, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - attrition_output_accuracy: 0.8013 - department_output_accuracy: 0.5783 - loss: 1.4965 - val_attrition_output_accuracy: 0.7873 - val_department_output_accuracy: 0.6063 - val_loss: 1.4060
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attrition_output_accuracy: 0.8370 - department_output_accuracy: 0.6645 - loss: 1.1898 - val_attrition_output_accuracy: 0.7873 - val_department_output_accuracy: 0.6063 - val_loss: 1.3413
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8437 - department_output_accuracy: 0.6491 - loss: 1.1673 - val_attrition_output_accuracy: 0.8009 - val_department_output_accuracy: 0.6063 - val_loss: 1.3339
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_accuracy: 0.8401 - department_output_accuracy: 0.6633 - loss: 1.1572 - val_attrition_output_accuracy: 0.7964 - val_department_output_accuracy: 0.6063 - val_loss: 1.3400
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7

In [155]:
# Evaluate the model with the testing data
attrition_loss, attrition_accuracy, department_accuracy = model.evaluate(X_test_scaled, {'attrition_output': y_attrition_test_encoded, 'department_output': y_department_test_encoded})

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8244 - department_output_accuracy: 0.5505 - loss: 3.6076 


In [156]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {attrition_accuracy}")
print(f"Department Accuracy: {department_accuracy}")

Attrition Accuracy: 0.804347813129425
Department Accuracy: 0.529891312122345


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2. I used a sigmoid activation function for the 'Attrition' output layer as it contained binary values (Yes and No) which a sigmoid measures. For the 'Department' output layer, I used the 'softmax' activation function due to this output layer having three values instead of two, which means it needed a multi-class activation function.
3. One possible improvement might be to split 'y_df' into two distinct variables and then encode that data before running a training/test split. That way you can train/test the values for those features separately. Another improvement might be to add in additional features. We only used ten for this model, and by adding more, we might be able to improve its ability to accurately predict.